In [7]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [55]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_hpg.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
df.head(3)

,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
HPG,20210305,45.65,46.10,45.1,46.0,21396800,45.65,46.10,45.1,46.0,1390000,3463100,1818700,2021-03-05,9,2021,2021-9
HPG,20210304,46.90,46.90,45.0,45.9,27962100,46.90,46.90,45.0,45.9,650000,1975100,1620500,2021-03-04,9,2021,2021-9
HPG,20210303,46.55,47.05,46.5,46.8,22262900,46.55,47.05,46.5,46.8,715000,1295500,979400,2021-03-03,9,2021,2021-9


In [52]:
df_hpg =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df_hpg = df_hpg.head(600)

In [53]:
#plot linechart for HPG price
short_rolling = df_hpg['<Close>'].rolling(window=20).mean()

ema_long = df_hpg['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df_hpg['<Close>'].ewm(span=20, adjust=False).mean()
line_hpg = df_hpg['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})

fig.add_trace(go.Scatter(x=df_hpg.datetime, y=line_hpg, 
                        mode='lines', name='HPG'))

fig.add_trace(go.Scatter(x=df_hpg.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df_hpg.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [54]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df_hpg,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df_hpg.columns)